In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from vecstack import stacking

In [76]:
#Reading the data from theonion vs nottheonion
df = pd.read_csv('C:/Users/ASUS/Desktop/data/scrapedtweets/Tweets.csv')
df.sample(5)

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries,Label
262,1196513648929378305,Mon Nov 18 19:40:58 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",RT @djordjepadejski: Serbian reporter @miodrag...,Serbian reporter accused causing cardiovascula...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,49,TamaraSinobad,NaN,NaN,"djordjepadejski, miodrag_sovilj, avucic",NaN,NaN,0
278,1196478851175649281,Mon Nov 18 17:22:42 +0000 2019,"<a href=""http://twitter.com/download/android"" ...",RT @djordjepadejski: Serbian reporter @miodrag...,Serbian reporter accused causing cardiovascula...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,49,1od5milionaV1,NaN,NaN,"djordjepadejski, miodrag_sovilj, avucic",NaN,NaN,0
120,1197467032637313024,Thu Nov 21 10:49:23 +0000 2019,"<a href=""http://twitter.com/#!/download/ipad"" ...",These need to start coming with a #NotTheOnion...,These need start coming watermark hashtag,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,0,Drizztdj,False,NotTheOnion,NaN,Minneapolis,NaN,0
142,1197345596400656389,Thu Nov 21 02:46:50 +0000 2019,"<a href=""Http://www.youtbe.com"" rel=""nofollow""...","RT @BryanDawsonUSA: Meanwhile, Fox News watche...",Meanwhile Fox News watchers debating movie Fro...,"Sentiment(polarity=0.2, subjectivity=0.2)",0.2,0.2,en,0,9,RealBryanDawson,NaN,NaN,BryanDawsonUSA,NaN,NaN,0
285,1196474459428810752,Mon Nov 18 17:05:15 +0000 2019,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Serbian reporter @miodrag_sovilj accused of ca...,Serbian reporter accused causing cardiovascula...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,173,49,djordjepadejski,False,NaN,"miodrag_sovilj, avucic","Stanford, CA",NaN,0


In [77]:
#splitting test-train-x-y-Features
y = df[['Label']]
X = df.iloc[:,[1, 4, 6, 7, 9, 10, 11, 12, 13, 14, 15]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [78]:
#we will do an lstm on only the text of the data this corresponds to Xdl
Xdl = df.iloc[:,4]

In [79]:
#Same thing splits...
Xdl_train, Xdl_test, ydl_train, ydl_test = train_test_split(Xdl, y, test_size=0.2, random_state=0)

In [80]:
#import for the lstm
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [81]:
#converting text to sequences could be later optimized with word2vec or something
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(Xdl_train)
sequences = tok.texts_to_sequences(Xdl_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [61]:
#Defining our network 
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [62]:
#RMSprop is op asf
modeldl = RNN()
modeldl.summary()
modeldl.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_9 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [63]:
#meh
modeldl.fit(sequences_matrix,ydl_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 224 samples, validate on 56 samples
Epoch 1/10
224/224 [==============================] - 2s 8ms/step - loss: 0.6865 - acc: 0.5804 - val_loss: 0.6517 - val_acc: 0.8036
Epoch 2/10
224/224 [==============================] - 0s 2ms/step - loss: 0.6222 - acc: 0.8214 - val_loss: 0.4868 - val_acc: 0.8036
Epoch 3/10
224/224 [==============================] - 0s 2ms/step - loss: 0.5473 - acc: 0.8214 - val_loss: 0.5102 - val_acc: 0.8036


In [64]:
#Testing
test_sequences = tok.texts_to_sequences(Xdl_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = model.evaluate(test_sequences_matrix,ydl_test)
print('First model\n Test shit\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

70/70 [==============================] - 0s 1ms/step
First model
 Test shit
  Loss: 0.468
  Accuracy: 0.886


In [71]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train['created_at'])
sequences = tok.texts_to_sequences(X_train['created_at'])
X_train['created_at'] = sequence.pad_sequences(sequences,maxlen=max_len)

AttributeError: 'int' object has no attribute 'lower'

In [58]:
models = [
    KNeighborsClassifier(n_neighbors=5,
                        n_jobs=-1),
        
    RandomForestClassifier(random_state=0, n_jobs=-1, 
                           n_estimators=100, max_depth=3),
        
    XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                  n_estimators=100, max_depth=3),
]

In [74]:
X_train['created_at']

264    Mon Nov 18 18:55:49 +0000 2019
239    Tue Nov 19 00:19:18 +0000 2019
304    Mon Nov 25 18:03:28 +0000 2019
21     Tue Nov 26 22:28:41 +0000 2019
135    Thu Nov 21 03:38:58 +0000 2019
309    Mon Nov 25 10:45:50 +0000 2019
49     Mon Nov 25 16:52:01 +0000 2019
282    Mon Nov 18 17:13:05 +0000 2019
289    Mon Nov 18 14:52:03 +0000 2019
171    Wed Nov 20 18:28:07 +0000 2019
223    Tue Nov 19 06:33:07 +0000 2019
205    Tue Nov 19 15:46:08 +0000 2019
286    Mon Nov 18 15:28:25 +0000 2019
22     Tue Nov 26 22:19:11 +0000 2019
261    Mon Nov 18 20:04:08 +0000 2019
238    Tue Nov 19 00:21:30 +0000 2019
140    Thu Nov 21 03:08:42 +0000 2019
226    Tue Nov 19 05:33:14 +0000 2019
66     Sun Nov 24 21:24:33 +0000 2019
126    Thu Nov 21 05:11:33 +0000 2019
12     Wed Nov 27 03:50:12 +0000 2019
221    Tue Nov 19 06:34:05 +0000 2019
316    Fri Nov 22 21:15:44 +0000 2019
283    Mon Nov 18 17:11:03 +0000 2019
252    Mon Nov 18 21:30:58 +0000 2019
17     Tue Nov 26 22:59:20 +0000 2019
167    Wed N